In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.gridspec as gridspec
import matplotlib as mpl
from matplotlib import cm 
from matplotlib.colors import ListedColormap,LinearSegmentedColormap
import math
import matplotlib.tri as mtri
from mpl_toolkits.mplot3d import Axes3D

def h_dict(n,q):
    import numpy as np
    index = np.zeros((n**q,q))
    ar_ = np.arange(n)
    ar_re = np.flip(np.arange(n))
    for i in range(q):
        for j in range(n**(i+1)):
            if (j//n)%2 == 0:
                index[j*n**(q-i-1):j*n**(q-i-1)+n**(q-i-1),i] = ar_[j%n]
            elif (j//n)%2 == 1:
                index[j*n**(q-i-1):j*n**(q-i-1)+n**(q-i-1),i] = ar_re[j%n]
    h_dict = {}
    for k in range(index.shape[0]):
        h_dict["{}".format(index[k,:])] = k
   
    return h_dict

def n_space(n, val):
    import numpy
    bins = np.linspace(0,1,n+1)
    which = np.digitize(val, bins=bins)
    if val == 1:
        return n-1    return which-1

def find_z(n,ar):
    import numpy
    n_dim = ar.shape[0]
    sum_ar = 0
    for i in range(0,n_dim):
        sum_ar+=ar[-i]*(n**i)
    bins = np.linspace(0,n**n_dim+1,n**n_dim+1)
    which = np.digitize(sum_ar, bins=bins)
    return which

def freq_dict(z):
    freq_dict = {}
    for line in z:
        if "{}".format(line) in freq_dict:
            freq_dict["{}".format(line)] += 1
        else:
            freq_dict["{}".format(line)] = 1
    return freq_dict

def freq_arr(data ,dict_):
    length = data.shape[0]
    freq_ar = np.zeros((1,length))
    for i in range(0,length):
        freq_ar[0,i] = dict_["{}".format(data[i,:])]
    return freq_ar

def make_xy(n,sq):
    x = []
    for i in range(0,n**(sq+1)+1):
        x+=[i]
        if i>=1:
            x+=[i]
    x = x[:-1]
    y = []
    for i in range(0,n):
        y+=[i]
        y+=[i]
    y_r = y.copy()
    y_r.reverse()
    return x, y, y_r

def make_seq_colormap(rgba_list):
    rgba_list = list(rgba_list)
    N = 256
    color_ = np.ones((N, 4))
    color_[:, 0] = np.linspace(rgba_list[0], 1, N)
    color_[:, 1] = np.linspace(rgba_list[1], 1, N)
    color_[:, 2] = np.linspace(rgba_list[2], 1, N)
    color__cmp = ListedColormap(color_)
    color__cmp = ListedColormap(color__cmp.colors[::-1])
    return color__cmp

def make_xy(n,sq):
    x = []
    for i in range(0,n**(sq+1)+1):
        x+=[i]
        if i>=1:
            x+=[i]
    x = x[:-1]
    y = []
    for i in range(0,n):
        y+=[i]
        y+=[i]
    y_r = y.copy()
    y_r.reverse()
    return x, y, y_r

def short_proj():
    return np.dot(Axes3D.get_proj(ax), scale)

In [ ]:
algo = "algorithm_name"
fake_datas = np.loadtxt("Input_data_path".format(algo), delimiter=",", dtype='float32')
algo+= "_YS"
fake_datas = fake_datas[fake_datas[:,-2].argsort(kind='mergesort')]
fake_data = fake_datas[:,:16]
fake_datay = fake_datas[:,16:]
fake_dup = np.zeros((fake_data.shape[0],fake_data.shape[1]))
n = 10
pad = [4,8,12,16]
q=pad[0]
h_dict_ = h_dict(n,q)
for i in range(fake_data.shape[0]):
    for j in range(fake_data.shape[1]):
        fake_dup[i,j] = n_space(n,fake_data[i,j])

z = np.zeros((fake_data.shape[0],len(pad)))
start = 0
for loc in enumerate(pad):
    z_ar = fake_dup[:,start:loc[1]]
    print(fake_data[:,start:loc[1]])
    start = loc[1]
    lin_ = 0
    for line in z_ar:
        try: 
            z[lin_,loc[0]] = h_dict_["{}".format(line)]
        except KeyError:
            z[lin_,loc[0]] = h_dict_["{}".format(str(line).replace("[","[0. "))]
        lin_+=1

freqs = freq_dict(z)
freqs_ar = freq_arr(z, freqs)
z_freq = np.concatenate((z,freqs_ar.T),axis=1)
z_freq = z_freq[np.argsort(z_freq[:, -1])]
z = z_freq[:,:-1]
freqs_ar = z_freq[:,-1]
#ar_01 = np.zeros((z_ar.shape[0],1))+0.25

plt.rcParams['axes.linewidth'] = 2
plt.rcParams['xtick.bottom'] = False
plt.rcParams['xtick.labelbottom'] = False
plt.rcParams['ytick.left'] = False
plt.rcParams['ytick.labelleft'] = False

colors = ["#668aba"] #YS
colors = ["#66ba8a"] #UTS
rgba = []
[rgba.append(mpl.colors.to_rgba("{}".format(c),1)) for c in colors]
cmap = make_seq_colormap(rgba[0])

for x in range(1,4):
    for y in range(2,5):
        if x != y and x < y:
            fig = plt.figure(figsize = (5, 5))
            spec = gridspec.GridSpec(ncols=1, nrows=1, width_ratios=[1])
            ax = fig.add_subplot(spec[0])
            ax.set_aspect('equal', 'box')
            ax.set_xlim(0, n**4)
            ax.set_ylim(0, n**4)
            SC = ax.scatter(z[:,x-1],z[:,y-1], c = ar_01, cmap=cmap, vmin=0, vmax=1)
            #SC = ax.scatter(z[:,x-1],z[:,y-1], c = fake_datay[:,1], cmap=cmap, vmin=0, vmax=1)
            plt.savefig('save_path'.format(algo, x, y), dpi=600)

In [ ]:
plt.rcParams['axes.linewidth'] = 0
#plt.rcParams['axes.edgecolor'] = "#ecf3f9"

fig = plt.figure(figsize = (5, 0.4))
c = ["darkorange", "gold", "darkgoldenrod", "darkolivegreen"]
n = 10
sq = 3
linewidty = [1.6, 1.2, 0.8, 0.4]

for q in range(sq):
    if q == 0:
        x,y,y_r = make_xy(n,q)
        len_ = len(x)
        plt.step(x, y , label='', c = c[q], linewidth = linewidty[q], alpha = 1)
    elif q > 0:
        x,y,y_r = make_xy(n,q)
        for i in range(n**q):
            x_ = x[i*len_: i*len_+len_]
            x_ = [j / n**q for j in x_]
            if i%2 == 1:
                y_ = [j + q*(n-1) + 7*q for j in y_r]
            else:
                y_ = [j + q*(n-1) + 7*q for j in y]
            plt.step(x_, y_ , label='pre (default)', c = c[q], linewidth = linewidty[q])

plt.tick_params(axis = "both", bottom = False, top = False,left = False, labelleft = 0, labelbottom = 0)
plt.xlim(0,10)
plt.savefig("save_path".format(n,sq), bbox_inches='tight', dpi=600)
plt.show()

In [ ]:
algo = "algorithm_name"
fake_datas = np.loadtxt("Input_data_path".format(algo), delimiter=",", dtype='float32')

dim = 3

dict_5 = h_dict(5, 5)
dict_6 = h_dict(5, 6)

n = 5
fake_data = fake_datas[:,:16]
fake_datay = fake_datas[:,16:]
fake_dup = np.zeros((fake_data.shape[0],fake_data.shape[1]))

for i in range(fake_data.shape[0]):
    for j in range(fake_data.shape[1]):
        fake_dup[i,j] = n_space(n,fake_data[i,j])
z1 = fake_dup[:,:5]
z2 = fake_dup[:,5:10]
z3 = fake_dup[:,10:16]
z = np.zeros((fake_data.shape[0],3))

for i in range(fake_dup.shape[0]):
    z[i, 0] = dict_5["{}".format(z1[i,:])]
    z[i, 1] = dict_5["{}".format(z2[i,:])]
    z[i, 2] = dict_6["{}".format(z3[i,:])]

mpl.rcParams['legend.fontsize'] = 10

fig = plt.figure()
fig.patch.set_facecolor('white')
ax = fig.add_subplot(1,1,1, projection='3d')
ax.set_xlim3d(0, 5**5)
ax.set_ylim3d(0, 5**5)
ax.set_zlim3d(0, 5**6)

ax.set_xlabel('$Z_1$', fontsize=15 ,fontweight = "bold")
ax.set_ylabel('$Z_2$', fontsize=15, fontweight = "bold")
ax.set_zlabel('$Z_3$', fontsize=15, fontweight = "bold")

ax.scatter(z[:,0], z[:,1], z[:,2], marker='o',label=".", s=fake_datay[:,-1]*36, c = fake_datay[:,-1], cmap = plt.cm.Blues, linewidth = 0)

x_scale=1
y_scale=1
z_scale=3

scale=np.diag([x_scale, y_scale, z_scale, 1.0])
scale=scale*(1.0/scale.max())
scale[3,3]=1.0


ax.get_proj=short_proj
for line in ax.xaxis.get_ticklines():
    line.set_visible(False)
for line in ax.yaxis.get_ticklines():
    line.set_visible(False)
for line in ax.zaxis.get_ticklines():
    line.set_visible(False)

ax.xaxis.set_ticklabels([])
ax.yaxis.set_ticklabels([])
ax.zaxis.set_ticklabels([])

ax.view_init(elev=60., azim=20)
ax.set_rasterized(True)
plt.savefig('save_path', dpi=600, bbox_inches = "tight")